----
<a id='CoMD'></a>

## CoMD

Classical Molecular Dynamics algorithms and workloads

**Links:**  
[Doxygen Documentation](http://exmatex.github.io/CoMD/doxygen-mpi/index.html)  
[github](https://github.com/exmatex/CoMD)  
[Lab Home Page](http://www.exmatex.org/comd.html)  

Flags: `-std=c99 -DDouble -g -O5 -lm`

**Clang hpcrun (on Intel(R) Xeon(R) CPU E5-2699 v4 @ 2.20GHz):**  
  
CPUTIME: 7.11e+06  
95.2% in `ljForce(SimFlat* SimFlat*)` function within `ljForce.c`  
92.6% Total Time in `for (int iOff=iBox*MAXATOMS,ii=0; ii<nIBox; ii++,iOff++)` loop, line #185  

```c 
            for (int jOff=MAXATOMS*jBox,ij=0; ij<nJBox; ij++,jOff++)     //% Total CPU Time
            {
               real_t dr[3];
               int jId = s->atoms->gid[jOff];  
               if (jBox < s->boxes->nLocalBoxes && jId <= iId )             //10.7%
                  continue; // don't double count local-local pairs.
               real_t r2 = 0.0;
               for (int m=0; m<3; m++)
               {
                  dr[m] = s->atoms->r[iOff][m]-s->atoms->r[jOff][m];        //17.0%
                  r2+=dr[m]*dr[m];                                          //13.2%
               }

               if ( r2 > rCut2) continue;                                   //10.1%

               // Important note:
               // from this point on r actually refers to 1.0/r
               r2 = 1.0/r2;
               real_t r6 = s6 * (r2*r2*r2);
               real_t eLocal = r6 * (r6 - 1.0) - eShift;
               s->atoms->U[iOff] += 0.5*eLocal;                             //11.2%
               s->atoms->U[jOff] += 0.5*eLocal;

               // calculate energy contribution based on whether
               // the neighbor box is local or remote
               if (jBox < s->boxes->nLocalBoxes)
                  ePot += eLocal;
               else
                  ePot += 0.5 * eLocal;

               // different formulation to avoid sqrt computation
               real_t fr = - 4.0*epsilon*r6*r2*(12.0*r6 - 6.0);
               for (int m=0; m<3; m++)
               {
                  s->atoms->f[iOff][m] -= dr[m]*fr;
                  s->atoms->f[jOff][m] += dr[m]*fr;                         //9.8%
               }
            } // loop over atoms in jBox
    
```

**Heavy Lines:**  
**10.7% | ljForce.c - 193 :** `if (jBox < s->boxes->nLocalBoxes && jId <= iId )`  
**17.0% | ljForce.c - 198 :** `dr[m] = s->atoms->r[iOff][m]-s->atoms->r[jOff][m];`  
**13.2% | ljForce.c - 199 :** `r2+=dr[m]*dr[m];`  
**10.1% | ljForce.c - 202 :** `if ( r2 > rCut2) continue;`  
**11.2% | ljForce.c - 209 :** `s->atoms->U[iOff] += 0.5*eLocal`  
** &nbsp; 9.8% | ljForce.c - 224 :** `s->atoms->f[jOff][m] += dr[m]*fr;`

 ---



 ### Level 1 Data Cache  Run
 1.00e+08 Level 1 Data Cache Misses. 
 1.38e+08 Load Instructions.  
 1.36e+08 Store Instructions.  
 Level 1 Data Cache Hit Rate: 63.5%

---  

### Memory Density ??  
1.67e+10 Load Instructions.  
2.88e+09 Store Instructions.   
4.02e+10 Total Cycles.   
(1.67 + .288) / 4.02 = .487  
48.7% Memory Density